In [2]:
import langchain

In [4]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:
from dotenv import load_dotenv, find_dotenv

In [6]:
load_dotenv(find_dotenv())

True

In [7]:
import os
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

Collection of Data


In [8]:
import bs4

In [9]:
loader=WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title","post-header")
        )
    )
)

In [13]:
docs=loader.load()

In [14]:
docs[0].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

In [15]:
llm=ChatGroq(model="llama3-8b-8192")

In [16]:
model_name="BAAI/bge-small-en"

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings

hf_embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)


C:\Users\srira\AppData\Local\Temp\ipykernel_21552\2907316330.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embedding = HuggingFaceEmbeddings(
c:\Users\srira\OneDrive\Documents\Python Scripts\envs\langchain_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\srira\OneDrive\Documents\Python Scripts\envs\langchain_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store dupli

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits=text_splitter.split_documents(docs)

In [25]:
vectorstore=FAISS.from_documents(documents=splits,embedding=hf_embedding)

In [26]:
retreiver=vectorstore.as_retriever()

In [27]:
prompt=hub.pull("rlm/rag-prompt")

c:\Users\srira\OneDrive\Documents\Python Scripts\envs\langchain_env\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [28]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])


In [29]:
from langchain_core.runnables import RunnablePassthrough


In [30]:
rag_chain=(
    {"context": retreiver, "question": RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [31]:
rag_chain.invoke("What is task decomposition?")

'Task decomposition is the process of breaking down complex tasks into smaller, manageable steps. It can be done through various methods, such as using simple prompting, task-specific instructions, or human inputs.'